In [1]:
# 컴퓨터는 텍스트보다 숫자를 더 잘 처리함 따라서 자연어 처리시 텍스트를
# 숫자로 바꾸는게 용이

# 이런 기법을 적용시키기 위한 첫 단계로 단어들을 고유한 정수에 맵핑 시키는
# 전처리 작업이 필요
# 맵핑(mapping) : 어떤 값을 다른 값에 대응시키는 과정
# ex) 어떤 텍스트 단어가 5000개 존재 -> 5000개 전체에 1번부터 5000까지
# 고유한 정수 or 인덱스 부여 apple->1 banana->2 ...

# 인덱스를 부여하는 방법: 랜덤으로 부여하기도 하는데 보통 빈도수에 따라
#     정렬한 뒤 부여 (많이 나올수록 앞으로)

In [2]:
# 1. 정수 인코딩(Integer Encoding)
# 인코딩(Encoding): 데이터를 컴퓨터가 이해할 수 있는 바이너리 형식으로 
#     변환하는 과정
    
# 단어에 정수를 부여하는 방법 중 1
# 1. 단어를 빈도수 순으로 정렬한 단어 집합 생성
# 2. 빈도수가 높은 순으로 낮은 숫자 부여

In [5]:
from nltk.tokenize import sent_tokenize #문장별로 토큰화
from nltk.tokenize import word_tokenize #단어별로 토큰화
from nltk.corpus import stopwords #불용어

raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

#문장 토큰화 sent_tokenize
sentences = sent_tokenize(raw_text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [6]:
vocab = {} #단어 딕셔너리 생성
preprocessed_sentences = [] 
stop_words = set(stopwords.words('english'))

for sentence in sentences:
    #단어 토큰화
    tokenized_sentence = word_tokenize(sentence)
    result=[]
    
    for word in tokenized_sentence:
        word = word.lower()# 모든 문자열 소문자 변환
        
        if word not in stop_words:#불용어 탐지
            
            if len(word) > 2:#단어 길이 2이하 제거
                
                result.append(word)#result 리스트에 추가
                if word not in vocab: #단어가 딕셔너리에 없을시 추가
                    vocab[word] = 0
                vocab[word] += 1 #나온 만큼 빈도수 증가
    preprocessed_sentences.append(result) #문장마다 남은 단어들 표시
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [13]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [14]:
print(vocab['barber'])

8


In [19]:
#빈도수 높은순으로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse=True)
# .items(): 키 밸류값 쌍으로 얻기
# lambda: 함수 한줄요약
#  key = lambda x:x[1]: 밸류값을 기준으로 내림차순(reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [20]:
word_to_index = {} #딕셔너리 생성
i = 0
for (word, frequency) in vocab_sorted: #키, 밸류값 뽑기
    if frequency > 1: #빈도수 1 단어 제외
        i = i + 1 #빈도수 높은순으로 낮은숫자 부여
        word_to_index[word] = i
        
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [25]:
#상위 5개 단어만 사용
vocab_size = 5 

word_frequency = [word for word, index in word_to_index.items() 
                 if index >= vocab_size + 1] #6,7 번째 인덱스들만 추출

#해당 단어에 대한 인덱스 정보 삭제
for w in word_frequency:#추출 인덱스들만 원래 딕셔너리에서 삭제
    del word_to_index[w]
print(word_to_index)


{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
[]


In [26]:
# sentences의 첫 문장은 ['barber', 'person'] 인데 이는 [1,5]로 인코딩
# 두 번째 문장은 ['barber', 'good', 'person']인데 good이 매치되는 숫자가
# 없음 -> 이런 상황을 Out of vocabulary OOV라고 함

In [27]:
word_to_index['OOV'] = len(word_to_index) + 1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [28]:
encoded_sentences = []
for sentence in preprocessed_sentences:
    encoded_sentence = []
    for word in sentence:
        try:
            #단어 집합에 있는 단어일시 해당 단어의 정수 리턴
            encoded_sentence.append(word_to_index[word])
        except KeyError:
            #없으면 oov의 정수 리턴
            encoded_sentence.append(word_to_index['OOV'])
    encoded_sentences.append(encoded_sentence)#encoded_sentences에 추가
print(encoded_sentences)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


In [29]:
#Counter 사용
from collections import Counter
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [31]:
#하나의 단어리스트로 변환
all_word_list = sum(preprocessed_sentences,[])
print(all_word_list)


['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [32]:
#단어의 빈도수 카운트
vocab = Counter(all_word_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [33]:
print(vocab['barber']) #'barber'의 빈도수 리턴

8


In [34]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)#등장 빈도수가 높은 상위 vocab_size개의 단어만 저장
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [35]:
#빈도수가 높은 순으로 낮은 수 부여
word_to_index = {}
i = 0

for (word, index) in vocab:
    i +=1
    word_to_index[word] = i
    
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [39]:
#nltk에는 빈도수 계산도구인 FreqDist() 지원
from nltk import FreqDist
import numpy as np
vocab = FreqDist(np.hstack(preprocessed_sentences))# 대괄호, , 제거
print(vocab['barber'])


['barber' 'person' 'barber' 'good' 'person' 'barber' 'huge' 'person'
 'knew' 'secret' 'secret' 'kept' 'huge' 'secret' 'huge' 'secret' 'barber'
 'kept' 'word' 'barber' 'kept' 'word' 'barber' 'kept' 'secret' 'keeping'
 'keeping' 'huge' 'secret' 'driving' 'barber' 'crazy' 'barber' 'went'
 'huge' 'mountain']


In [40]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)#등장 빈도수가 높은 상위 vocab_size개의 단어만 저장
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [46]:
#enumerate 사용
word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
print(word_to_index)
#word[0]은 단어, word[1]은 인덱스값

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [47]:
# enumerate이해
test_input = ['a', 'b', 'c', 'd', 'e']
for index, value in enumerate(test_input):#입력순으로 0부터 인덱스 부여
    print("value : {}, index: {}".format(value, index))
    

value : a, index: 0
value : b, index: 1
value : c, index: 2
value : d, index: 3
value : e, index: 4


In [49]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [50]:
preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스 입력시 빈도수를 기준으로 단어집합 생성
# 빈도수가 높은순으로 낮은 정수 인덱스 부여
tokenizer.fit_on_texts(preprocessed_sentences)

In [51]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [53]:
print(tokenizer.word_counts)#빈도수를 보여줌

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [54]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))
#입력된 코퍼스에대해 이미 정해진 인덱스로 변환

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [56]:
#상위 5개의 단어만 사용
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1)#상위 5개 단어만 사용
tokenizer.fit_on_texts(preprocessed_sentences)

In [57]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [58]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [59]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

In [60]:
vocab_size = 5
words_frequency = [word for word, index in tokenizer.word_index.items() if index >= vocab_size + 1] 

# 인덱스가 5 초과인 단어 제거
for word in words_frequency:
    del tokenizer.word_index[word] # 해당 단어에 대한 인덱스 정보를 삭제
    del tokenizer.word_counts[word] # 해당 단어에 대한 카운트 정보를 삭제

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [61]:
# 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

In [62]:
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 : 1


In [63]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]
